In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# csv파일 병합 코드
bus_data = []

# 1. 2025년 데이터 (1월 ~ 12월)
for i in range(1,12):
    file_path = f'/content/drive/MyDrive/키키아이/data/원본/2025_{i}.csv'
    data = pd.read_csv(file_path, encoding='cp949')
    bus_data.append(data)


# 2. 2024년 데이터 (11월 ~ 12월)
for i in range(11,13):
    file_path = f'/content/drive/MyDrive/키키아이/data/원본/2024_{i}.csv'
    data = pd.read_csv(file_path, encoding='cp949')
    bus_data.append(data)

df = pd.concat(bus_data, ignore_index=True)

/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='cp949')
/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='cp949')
/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='cp949')
/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='cp949')
/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4,54) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='cp949')
/tmp/ipython-input-3100042322.py:8: DtypeWarning: Columns (4,54) ha

In [ ]:
print(df.isnull().sum()) # 결측치 x
print(df.describe(),'\n\n',df.describe(include='O'))


사용년월          0
노선번호          0
노선명           0
표준버스정류장ID     0
버스정류장ARS번호    0
역명            0
00시승차총승객수     0
00시하차총승객수     0
1시승차총승객수      0
1시하차총승객수      0
2시승차총승객수      0
2시하차총승객수      0
3시승차총승객수      0
3시하차총승객수      0
4시승차총승객수      0
4시하차총승객수      0
5시승차총승객수      0
5시하차총승객수      0
6시승차총승객수      0
6시하차총승객수      0
7시승차총승객수      0
7시하차총승객수      0
8시승차총승객수      0
8시하차총승객수      0
9시승차총승객수      0
9시하차총승객수      0
10시승차총승객수     0
10시하차총승객수     0
11시승차총승객수     0
11시하차총승객수     0
12시승차총승객수     0
12시하차총승객수     0
13시승차총승객수     0
13시하차총승객수     0
14시승차총승객수     0
14시하차총승객수     0
15시승차총승객수     0
15시하차총승객수     0
16시승차총승객수     0
16시하차총승객수     0
17시승차총승객수     0
17시하차총승객수     0
18시승차총승객수     0
18시하차총승객수     0
19시승차총승객수     0
19시하차총승객수     0
20시승차총승객수     0
20시하차총승객수     0
21시승차총승객수     0
21시하차총승객수     0
22시승차총승객수     0
22시하차총승객수     0
23시승차총승객수     0
23시하차총승객수     0
교통수단타입코드      0
교통수단타입명       0
등록일자          0
dtype: int64
                사용년월     표준버스정류장ID      00시승차총승객수      00시하차총승객수  \
count  

In [ ]:
# print(df.info()) # 결과 확인

# 3. 불필요한 컬럼('등록일자')제거
df = df.drop(columns='등록일자')

In [ ]:
# 4. 구조변환(wide -> long)

id_vars = ['사용년월', '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '교통수단타입코드', '교통수단타입명']
melted_df = df.melt(id_vars=id_vars, var_name='시간_구분', value_name='승객수')

# 문자 추출
split_data = melted_df['시간_구분'].str.extract(r'(\d+)시(승차|하차)총승객수')
# print(split_data)

melted_df['시간'] = split_data[0]       # 첫 번째 괄호 -> 시간
melted_df['승하차구분'] = split_data[1] # 두 번째 괄호 -> 승하차구분

melted_df['시간'] = melted_df['시간'].astype(int) # int로 변환

final_df = melted_df.drop(columns=['시간_구분'])
print("변환된 데이터 크기:", final_df.shape)
print(final_df.head())

변환된 데이터 크기: (26432544, 11)
     사용년월 노선번호                    노선명  표준버스정류장ID 버스정류장ARS번호              역명  \
0  202501  470       470번(상암차고지~안골마을)  100000001       1001  종로2가사거리(00066)   
1  202501  N37  N37번(진관공영차고지~송파공영차고지)  100000001       1001  종로2가사거리(00089)   
2  202501  470       470번(상암차고지~안골마을)  100000001       1001  종로2가사거리(00064)   
3  202501  741      741번(진관차고지~헌인릉입구)  100000001       1001  종로2가사거리(00073)   
4  202501  N37  N37번(송파공영차고지~진관공영차고지)  100000001       1001  종로2가사거리(00032)   

  교통수단타입코드 교통수단타입명  승객수  시간 승하차구분  
0       10  서울간선버스    6   0    승차  
1       51  서울심야버스    0   0    승차  
2       10  서울간선버스   45   0    승차  
3       10  서울간선버스   91   0    승차  
4       51  서울심야버스   11   0    승차  


In [ ]:
# 4. 전처리된 데이터 저장
# final_df.to_csv('/content/drive/MyDrive/키키아이/data/가공 데이터/final.csv')